In [1]:
from pandas_datareader import data as pdr
import datetime
import numpy as np
import matplotlib.pyplot as plt

C:\Users\Rohit Penumarti\Anaconda3\lib\site-packages\pandas_datareader\compat\__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [8]:
def StockVol(histoPrice):
    closing_prices = histoPrice['Adj Close'].to_numpy()
    returns = np.log(closing_prices[1:]/closing_prices[:-1])
    vol = np.sqrt(252)*np.sqrt(np.var(returns))
    return vol

In [9]:
def StockPath(n, sigma, S0, T, num_p, r):
    paths = np.zeros((n, num_p+1))
    paths[:, 0] = S0
    dt = T/num_p
    
    for i in range(n):
        for j in range(1, num_p+1):
            rand = np.random.standard_normal()
            paths[i, j] = paths[i, j-1]*np.exp((r-0.5*sigma**2)*dt+sigma*np.sqrt(dt)*rand)
    
    return paths

In [29]:
def EurOptPrice(n, paths, sigma, T, num_p, r, K):
    payoff = np.exp(-r*T)*np.maximum(K-paths[:, num_p], 0)
    price = np.average(payoff)
    var = np.var(payoff)
    return [payoff, price, var]

In [96]:
def AmeOptPrice(n, paths, sigma, T, num_p, r, K):
    V_hat = np.zeros((n, num_p))
    h = np.maximum(K-paths, 0)
    V_hat[:, num_p-1] = h[:, num_p-1]
    
    dt = T/num_p
    df = np.exp(-r*dt)
    
    for i in range(num_p-1, 0, -1):
        fit = np.polyfit(paths[:, i], V_hat[:, i]*df, 5)
        C = np.polyval(fit, paths[:, i])
        V_hat[:, i-1] = np.where(h[:, i] > C, h[:, i], V_hat[:, i]*df)
        
    price = np.sum(V_hat[:, 1]*df)/n
    var = np.var(V_hat[:, 1]*df)
    return [V_hat, price, var]

In [97]:
def ContVariate(n, paths, sigma, T, S0, num_p, r, K):
    V_hat = AmeOptPrice(n, paths, sigma, T, num_p, r, K)[0]
    
    dt = T/num_p
    df = np.exp(-r*dt)
    
    Y = V_hat[:, 1]*df
    X = paths[:, 1]*df

    EX = np.exp(r*(1/num_p))*S0
        
    X_mean = np.average(X)
    Y_mean = np.average(Y)
    b = np.sum((X-X_mean)*(Y-Y_mean))/np.sum((X-X_mean)**2)
    estimator = (1/n)*np.sum(Y-b*(X-EX))
    var = np.var(Y-b*(X-EX))
    
    return [estimator, var]

In [98]:
start = (datetime.datetime.today() - datetime.timedelta(days=366)).strftime('%m/%d/%Y')
end = datetime.datetime.today().strftime('%m/%d/%Y')
    
df = pdr.get_data_yahoo('GOOG', start=start, end=end)

vol = StockVol(df)
S0 = df['Adj Close'][0]
T = 1
num_p = 252
r = 0.0068
n = 1000

paths = StockPath(n, vol, S0, T, num_p, r)

K = 1440
priceEuro = EurOptPrice(n, paths, vol, T, num_p, r, K)[1:]
print("The price and variance, respectively, of the European put option with strike " + str(K) + " is: " + str(priceEuro))

priceAmer = AmeOptPrice(n, paths, vol, T, num_p, r, K)[1:]
print("The price and variance, respectively, of the American put option with strike " + str(K) + " is: " + str(priceAmer))

price_cont_var = ContVariate(n, paths, vol, T, S0, num_p, r, K)
print("The price and variance, respectively of the American put option with strike " + str(K) + " using control variates as "
      "a technique for variance reduction is: " + str(price_cont_var))

The price and variance, respectively, of the European put option with strike 1440 is: [387.06129841812725, 78697.7954687852]
The price and variance, respectively, of the American put option with strike 1440 is: [405.2627985361096, 39046.140040044134]
The price and variance, respectively of the American put option with strike 1440 using control variates as a technique for variance reduction is: [404.95384872066705, 38904.23296077818]
